# Global Carbon Dioxide Emission By Country: ETL

### Introduction 

This notebook contains the cleanup and exploration for the publicly available data of Global Carbon Dioxide Emission By Country. The results are reorganized CSV files and MongoDB collections that has data ready for analysis. The resulting data includes: 
* Annual Global CO2 Emissions for years 1970 through 2018 for a total of 208 countries 
* Annual Global CO2 Emissions for years 1970 through 2019 for a total of 66 countries 
       


### Data Extraction

* The Global Carbon Dioxide Emission By Country data comes from two sources: 
    * Knoema: https://knoema.com/EDGARED2019/global-ghg-and-co2-emissions 
        * 208 countries (1970-2018)
    * Nationmaster: https://www.nationmaster.com/nmx/ranking/carbon-dioxide-emissions
        * 66 countries (2019)
        
* The source of each country's population count  in 2019 is https://worldpopulationreview.com/ where the JSON file is provided 

* Data Provided includes:
    * Country
    * Fossil CO2 Emissions as Indicator 
    * Population
    * All Type: Power Industry, Transport, Buildings, Other Industrial Combustion, Other Sectors 
    * Frequency: Annually
    * Unit in Metric ton
    * Annual CO2 emission by country 

In [308]:
# Import Dependencies and Setup
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
import json
import knoema
import pymongo

# JSON: 2020 Population (by country) data extraction 

In this section, a JSON file containing population count for every country in 2019, downloaded from https://worldpopulationreview.com/, is read and converted into a dataframe. 

In [309]:
# Read Json file into dataframe
population = pd.read_json('../ETL_project/Data/population_data.json')
population

,Rank,name,pop2020,pop2019,GrowthRate,area,Density
0,1,China,1439323.776,1433783.686,1.0039,9706961.00,147.7068
1,2,India,1380004.385,1366417.754,1.0099,3287590.00,415.6290
2,3,United States,331002.651,329064.917,1.0059,9372610.00,35.1092
3,4,Indonesia,273523.615,270625.568,1.0107,1904569.00,142.0928
4,5,Pakistan,220892.340,216565.318,1.0200,881912.00,245.5634
...,...,...,...,...,...,...,...
227,228,Montserrat,4.992,4.989,1.0006,102.00,48.9118
228,229,Falkland Islands,3.480,3.377,1.0305,12173.00,0.2774
229,230,Niue,1.626,1.615,1.0068,260.00,6.2115
230,231,Tokelau,1.357,1.340,1.0127,12.00,111.6667


In [310]:
# Delete unwanted columns from population df
del population['Rank']
del population['GrowthRate']
del population['area']
del population['Density']
del population['pop2020']

# Rename columns
population_df =population.rename(columns={"name": "Location", "pop2019":"Population(2019)"})
population_df

,Location,Population(2019)
0,China,1433783.686
1,India,1366417.754
2,United States,329064.917
3,Indonesia,270625.568
4,Pakistan,216565.318
...,...,...
227,Montserrat,4.989
228,Falkland Islands,3.377
229,Niue,1.615
230,Tokelau,1.340


In [311]:
# Save as CSV
population_df.to_csv('../ETL_project/Data/CSV_data/2019population_csv')

# API: 1970 - 2018 CO2 data extraction

In this section, an API file containing the global CO2 emissions by country from https://knoema.com/EDGARED2019/global-ghg-and-co2-emissions is extracted by installing knoema. Since the dataset is open for public, no access requirement such as password is required. The site provides a line of code that includes a knoema.get() function for data extraction.

In [13]:
# Install knoema to extract api data from the site
# pip install knoema

SyntaxError: invalid syntax (<ipython-input-13-6d19c5370f20>, line 2)

In [312]:
# Retrieve 1st carbon emission dataset using python api provided by the knoema 
# Unit: Mton CO2
carbon_data = knoema.get('EDGARED2019', timerange='1970-2018', frequency='A', Location='US;AF;AL;DZ;AO;AI;AG;AR;AM;AW;AU;AT;AZ;BS;BH;BD;BB;BY;BE;BZ;BJ;BM;BT;BO;BA;BW;BR;VG;BN;BG;BF;BI;CV;KH;CM;CA;KY;CF;TD;CL;CN;CO;KM;CG;CD;CK;CR;CI;HR;CU;CW;CY;CZ;DK;DJ;DM;DO;EC;EG;SV;GQ;ER;EE;ET;FK;FO;FJ;FI;KN.FM;GF;PF;GA;GM;GE;DE;GH;GI;GR;GL;GD;GP;GT;GN;GW;GY;HT;HN;HK;HU;IS;IN;ID;IR;IQ;IE;KN.IP;KN.IS;JM;JP;JO;KZ;KE;KI;KP;KR;KW;KG;LA;LV;LB;LS;LR;LY;LT;LU;MO;MK;MG;MW;MY;MV;ML;MT;MQ;MR;MU;MX;MD;MN;MA;MZ;MM;NA;NP;NL;NC;NZ;NI;NE;NG;NO;OM;PK;PW;PA;PG;PY;PE;PH;PL;PT;PR;QA;RE;RO;RU;RW;KN.SH;KN;LC;PM;VC;WS;ST;SA;SN;KN.A2;SC;SL;SG;SK;SI;SB;SO;ZA;KN.SA;LK;KN.SS;SR;SZ;SE;KN.SL;SY;TW;TJ;TZ;TH;TL;TG;TO;TT;TN;TR;TM;TC;UG;UA;AE;GB;UY;UZ;VU;VE;VN;EH;YE;ZM;ZW', Indicator='KN.F1', Type='All')

# Display data extracted
print(carbon_data)

Location          United States          Afghanistan              Albania  \
Indicator  Fossil CO2 Emissions Fossil CO2 Emissions Fossil CO2 Emissions   
Type                   All Type             All Type             All Type   
Frequency                     A                    A                    A   
1970-01-01          4688.522395             1.466159             4.377893   
1971-01-01          4559.408967             1.465293             4.369894   
1972-01-01          4807.030715             1.510878             4.953242   
1973-01-01          4994.593321             1.473702             4.540402   
1974-01-01          4827.419694             1.895983             4.744196   
1975-01-01          4612.546577             1.693211             4.888888   
1976-01-01          4907.271162             1.609170             5.256256   
1977-01-01          5071.365566             1.868496             5.543410   
1978-01-01          5066.244675             1.614685             6.466902   

In [314]:
# Print column names to verify multiindex type
# list(carbon_data.columns.values)
print(carbon_data.columns)

MultiIndex([(      'United States', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (        'Afghanistan', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (            'Albania', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (            'Algeria', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (             'Angola', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (           'Anguilla', 'Fossil CO2 Emissions', 'All Type', 'A'),
            ('Antigua and Barbuda', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (          'Argentina', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (            'Armenia', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (              'Aruba', 'Fossil CO2 Emissions', 'All Type', 'A'),
            ...
            (     'United Kingdom', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (            'Uruguay', 'Fossil CO2 Emissions', 'All Type', 'A'),
            (         'Uzbekistan', 'Fossil CO2 

In [315]:
# Verify column types 
carbon_data.dtypes

Location        Indicator             Type      Frequency
United States   Fossil CO2 Emissions  All Type  A            float64
Afghanistan     Fossil CO2 Emissions  All Type  A            float64
Albania         Fossil CO2 Emissions  All Type  A            float64
Algeria         Fossil CO2 Emissions  All Type  A            float64
Angola          Fossil CO2 Emissions  All Type  A            float64
                                                              ...   
Vietnam         Fossil CO2 Emissions  All Type  A            float64
Western Sahara  Fossil CO2 Emissions  All Type  A            float64
Yemen           Fossil CO2 Emissions  All Type  A            float64
Zambia          Fossil CO2 Emissions  All Type  A            float64
Zimbabwe        Fossil CO2 Emissions  All Type  A            float64
Length: 208, dtype: object

In [316]:
# Reflect the DataFrame over its main diagonal by writing rows as columns and vice-versa
carbon_data_transposed = carbon_data.transpose()
carbon_data_transposed.head()

,,,,1970-01-01,1971-01-01,1972-01-01,1973-01-01,1974-01-01,1975-01-01,1976-01-01,1977-01-01,1978-01-01,1979-01-01,...,2009-01-01,2010-01-01,2011-01-01,2012-01-01,2013-01-01,2014-01-01,2015-01-01,2016-01-01,2017-01-01,2018-01-01
Location,Indicator,Type,Frequency,,,,,,,,,,,,,,,,,,,,,
United States,Fossil CO2 Emissions,All Type,A,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,4907.271162,5071.365566,5066.244675,5105.068431,...,5314.842145,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845
Afghanistan,Fossil CO2 Emissions,All Type,A,1.466159,1.465293,1.510878,1.473702,1.895983,1.693211,1.609170,1.868496,1.614685,1.741531,...,4.248142,5.641922,6.923214,9.662172,16.311808,10.329014,9.800058,9.938866,10.187785,10.476969
Albania,Fossil CO2 Emissions,All Type,A,4.377893,4.369894,4.953242,4.540402,4.744196,4.888888,5.256256,5.543410,6.466902,7.589628,...,4.308789,4.592990,4.988055,4.543900,4.657987,5.064173,4.709439,4.689464,4.888502,4.802468
Algeria,Fossil CO2 Emissions,All Type,A,18.925117,22.310483,34.368764,46.469252,39.294911,35.704459,46.316389,51.285003,56.554711,56.523209,...,114.677988,116.408456,122.761349,132.331115,135.762931,146.486768,156.269210,156.328906,157.330772,165.331981
Angola,Fossil CO2 Emissions,All Type,A,8.952848,8.538195,10.388669,11.371827,11.832747,10.929404,7.316239,12.056412,14.237736,14.202955,...,21.491206,24.018474,24.983535,25.221763,27.426121,29.764167,32.737569,33.548279,32.052138,30.047294


In [317]:
# Reset Dataframe Index
new_carbon_data=carbon_data_transposed.reset_index()
new_carbon_data

,Location,Indicator,Type,Frequency,1970-01-01 00:00:00,1971-01-01 00:00:00,1972-01-01 00:00:00,1973-01-01 00:00:00,1974-01-01 00:00:00,1975-01-01 00:00:00,...,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00
0,United States,Fossil CO2 Emissions,All Type,A,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,...,5314.842145,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845
1,Afghanistan,Fossil CO2 Emissions,All Type,A,1.466159,1.465293,1.510878,1.473702,1.895983,1.693211,...,4.248142,5.641922,6.923214,9.662172,16.311808,10.329014,9.800058,9.938866,10.187785,10.476969
2,Albania,Fossil CO2 Emissions,All Type,A,4.377893,4.369894,4.953242,4.540402,4.744196,4.888888,...,4.308789,4.592990,4.988055,4.543900,4.657987,5.064173,4.709439,4.689464,4.888502,4.802468
3,Algeria,Fossil CO2 Emissions,All Type,A,18.925117,22.310483,34.368764,46.469252,39.294911,35.704459,...,114.677988,116.408456,122.761349,132.331115,135.762931,146.486768,156.269210,156.328906,157.330772,165.331981
4,Angola,Fossil CO2 Emissions,All Type,A,8.952848,8.538195,10.388669,11.371827,11.832747,10.929404,...,21.491206,24.018474,24.983535,25.221763,27.426121,29.764167,32.737569,33.548279,32.052138,30.047294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Vietnam,Fossil CO2 Emissions,All Type,A,13.407028,13.342814,14.358230,14.535535,9.833613,9.885531,...,138.730752,154.679966,154.562733,153.242675,159.428876,174.941642,202.770459,242.065191,242.460757,271.474096
204,Western Sahara,Fossil CO2 Emissions,All Type,A,0.073311,0.073310,0.072442,0.074264,0.078865,0.082973,...,0.258750,0.265536,0.312064,0.325751,0.267194,0.242155,0.253520,0.258331,0.273070,0.283864
205,Yemen,Fossil CO2 Emissions,All Type,A,1.501672,1.514800,1.792100,2.209955,2.195514,2.102490,...,29.024687,27.175280,22.845059,21.518648,28.352236,26.911149,14.286612,11.410842,11.542397,11.767991
206,Zambia,Fossil CO2 Emissions,All Type,A,3.669060,3.826292,4.234740,4.539008,4.561470,4.732814,...,2.036439,2.239055,2.552793,3.579864,3.801032,4.275357,4.442017,4.700361,5.054812,5.166280


In [318]:
# Check index 
print(new_carbon_data.index)

RangeIndex(start=0, stop=208, step=1)


In [320]:
# Check column data types
new_carbon_data.dtypes

Location                object
Indicator               object
Type                    object
Frequency               object
1970-01-01 00:00:00    float64
1971-01-01 00:00:00    float64
1972-01-01 00:00:00    float64
1973-01-01 00:00:00    float64
1974-01-01 00:00:00    float64
1975-01-01 00:00:00    float64
1976-01-01 00:00:00    float64
1977-01-01 00:00:00    float64
1978-01-01 00:00:00    float64
1979-01-01 00:00:00    float64
1980-01-01 00:00:00    float64
1981-01-01 00:00:00    float64
1982-01-01 00:00:00    float64
1983-01-01 00:00:00    float64
1984-01-01 00:00:00    float64
1985-01-01 00:00:00    float64
1986-01-01 00:00:00    float64
1987-01-01 00:00:00    float64
1988-01-01 00:00:00    float64
1989-01-01 00:00:00    float64
1990-01-01 00:00:00    float64
1991-01-01 00:00:00    float64
1992-01-01 00:00:00    float64
1993-01-01 00:00:00    float64
1994-01-01 00:00:00    float64
1995-01-01 00:00:00    float64
1996-01-01 00:00:00    float64
1997-01-01 00:00:00    float64
1998-01-

In [319]:
# Save extracted data as a CSV
new_carbon_data.to_csv('../ETL_project/Data/CSV_data/carbon1970-2018_csv')

# Web Scraping: 2019 CO2 data extraction

In this section, the 2019 CO2 emission data is scraped from https://www.nationmaster.com/nmx/ranking/carbon-dioxide-emissions. The data is only available for 66 countries unlike the previous datasets obtained. 

In [33]:
# # Set executable path & assign Chrome Browser
# executable_path = {'executable_path': './chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

In [34]:
# # Visit URL through splinter
# url = 'https://www.nationmaster.com/nmx/ranking/carbon-dioxide-emissions'
# browser.visit(url)

In [35]:
# # HTML Object
# html = browser.html

# # Parse HTML with Beautiful Soup
# soup = BeautifulSoup(html, 'html.parser')

In [324]:
# Visit the Site Using Pandas to Read
carbon_2019_df = pd.read_html('https://www.nationmaster.com/nmx/ranking/carbon-dioxide-emissions')[0]
print(carbon_2019_df)

# Assign the columns amd set index
carbon_2019_df.columns=["Rank", "Location", "2019-01-01 00:00:00", "Year", "YoY", "5-years CAGR", "View"]

# carbon_2019_df.set_index("Description")
carbon_2019_df

     #              66 Countries  Million Metric Tons  Last     YoY  \
0    1                 #1  China              9622.14  2019  +2.0 %   
1    2         #2  United States              5076.20  2019  -1.3 %   
2    3                 #3  India              2525.08  2019  +1.9 %   
3    4                #4  Russia              1530.60  2019  -1.3 %   
4    5                 #5  Japan              1150.06  2019  +0.1 %   
..  ..                       ...                  ...   ...     ...   
61  62              #62  Denmark                34.08  2019  -4.4 %   
62  63             #63  Slovakia                32.29  2019  -1.7 %   
63  64           #64  Azerbaijan                32.14  2019  +1.2 %   
64  65  #65  Trinidad and Tobago                20.96  2019  +1.2 %   
65  66            #66  Lithuania                11.94  2019  -1.7 %   

   5âyears CAGR Unnamed: 6  
0          +0.8 %  View data  
1          -0.9 %  View data  
2          +3.9 %  View data  
3          +0.0 %  View

,Rank,Location,2019-01-01 00:00:00,Year,YoY,5-years CAGR,View
0,1,#1 China,9622.14,2019,+2.0 %,+0.8 %,View data
1,2,#2 United States,5076.20,2019,-1.3 %,-0.9 %,View data
2,3,#3 India,2525.08,2019,+1.9 %,+3.9 %,View data
3,4,#4 Russia,1530.60,2019,-1.3 %,+0.0 %,View data
4,5,#5 Japan,1150.06,2019,+0.1 %,-1.5 %,View data
...,...,...,...,...,...,...,...
61,62,#62 Denmark,34.08,2019,-4.4 %,-3.2 %,View data
62,63,#63 Slovakia,32.29,2019,-1.7 %,+1.6 %,View data
63,64,#64 Azerbaijan,32.14,2019,+1.2 %,+0.7 %,View data
64,65,#65 Trinidad and Tobago,20.96,2019,+1.2 %,-1.5 %,View data


In [325]:
# Delete unwanted columns 
del carbon_2019_df['Rank']
del carbon_2019_df['YoY']
del carbon_2019_df['5-years CAGR']
del carbon_2019_df['View']
del carbon_2019_df['Year']

In [326]:
# Set Location as the index
carbon_2019_df.set_index('Location', inplace=True)
carbon_2019_df

,2019-01-01 00:00:00
Location,
#1 China,9622.14
#2 United States,5076.20
#3 India,2525.08
#4 Russia,1530.60
#5 Japan,1150.06
...,...
#62 Denmark,34.08
#63 Slovakia,32.29
#64 Azerbaijan,32.14


In [327]:
# Rename each index value names
carbon2019_renamed_df = carbon_2019_df.rename(
    index=
    {
        '#1  China': "China", '#2  United States': 'United States', '#3  India': 'India', '#4  Russia': 'Russia',
       '#5  Japan': 'Japan', '#6  Germany': 'Germany', '#7  South Korea': 'South Korea', '#8  Iran': 'Iran',
       '#9  Saudi Arabia': 'Saudi Arabia', '#10  Canada': 'Canada', '#11  Indonesia': 'Indonesia', '#12  Mexico':'Mexico',
       '#13  Brazil': 'Brazil', '#14  South Africa': 'South Africa', '#15  Australia': 'Australia', '#16  Turkey': 'Turkey',
       '#17  United Kingdom': 'United Kingdom', '#18  Italy': 'Italy', '#19  Poland': 'Poland', '#20  France':'France',
       '#21  Thailand':'Thailand', '#22  Spain':'Spain', '#23  Taiwan':'Taiwan',
       '#24  United Arab Emirates':'United Arab Emirates', '#25  Malaysia':'Malaysia', '#26  Kazakhstan':'Kazakhstan',
       '#27  Singapore':'Singapore', '#28  Egypt':'Egypt', '#29  Vietnam':'Vietnam', '#30  Netherlands':'Netherlands',
       '#31  Pakistan':'Pakistan', '#32  Argentina':'Argentina', '#33  Ukraine':'Ukraine', '#34  Algeria':'Algeria',
       '#35  Philippines':'Philippines', '#36  Belgium':'Belgium', '#37  Venezuela':'Venezuela', '#38  Qatar':'Qatar',
       '#39  Uzbekistan':'Uzbekistan', '#40  Czech Republic':'Czech Republic', '#41  Kuwait':'Kuwait',
       '#42  Colombia':'Colombia', '#43  Chile':'Chile', '#44  Bangladesh':'Bangladesh', '#45  Turkmenistan':'Turkmenistan',
       '#46  Greece':'Greece', '#47  Israel':'Israel', '#48  Romania':'Romania', '#49  Austria':'Austria',
       '#50  Belarus':'Belarus', '#51  Portugal':'Portugal', '#52  Peru':'Peru', '#53  Hungary':'Hungary',
       '#54  Finland':'Finland', '#55  Sweden':'Sweden', '#56  Bulgaria':'Bulgaria', '#57  Ireland':'Ireland',
       '#58  Ecuador':'Ecuador', '#59  Switzerland':'Switzerland', '#60  New Zealand':'New Zealand', '#61  Norway':'Norway',
       '#62  Denmark':'Denmark', '#63  Slovakia':'Slovakia', '#64  Azerbaijan':'Azerbaijan',
       '#65  Trinidad and Tobago':'Trinidad and Tobago', '#66  Lithuania':'Lithuania'
    }
)
carbon2019_renamed_df

,2019-01-01 00:00:00
Location,
China,9622.14
United States,5076.20
India,2525.08
Russia,1530.60
Japan,1150.06
...,...
Denmark,34.08
Slovakia,32.29
Azerbaijan,32.14


In [328]:
# Reset index 
carbon2019_renamed_df.reset_index()

,Location,2019-01-01 00:00:00
0,China,9622.14
1,United States,5076.20
2,India,2525.08
3,Russia,1530.60
4,Japan,1150.06
...,...,...
61,Denmark,34.08
62,Slovakia,32.29
63,Azerbaijan,32.14
64,Trinidad and Tobago,20.96


In [329]:
# Save as CSV 
carbon2019_renamed_df.to_csv('../ETL_project/Data/CSV_data/carbon2019_csv')

# Data Exploration, Transformation, and Cleanup

This section includes exploring, transforming, and cleaning up the data if necessary. There are no missing values in the dataset based on the data exploration done. 

In [330]:
# Read saved CSV 1970-2018 data
carbon_data_df = pd.read_csv('../ETL_project/Data/CSV_data/carbon1970-2018_csv', index_col=0)
carbon_data_df.head()

,Location,Indicator,Type,Frequency,1970-01-01 00:00:00,1971-01-01 00:00:00,1972-01-01 00:00:00,1973-01-01 00:00:00,1974-01-01 00:00:00,1975-01-01 00:00:00,...,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00
0,United States,Fossil CO2 Emissions,All Type,A,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,...,5314.842145,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845
1,Afghanistan,Fossil CO2 Emissions,All Type,A,1.466159,1.465293,1.510878,1.473702,1.895983,1.693211,...,4.248142,5.641922,6.923214,9.662172,16.311808,10.329014,9.800058,9.938866,10.187785,10.476969
2,Albania,Fossil CO2 Emissions,All Type,A,4.377893,4.369894,4.953242,4.540402,4.744196,4.888888,...,4.308789,4.592990,4.988055,4.543900,4.657987,5.064173,4.709439,4.689464,4.888502,4.802468
3,Algeria,Fossil CO2 Emissions,All Type,A,18.925117,22.310483,34.368764,46.469252,39.294911,35.704459,...,114.677988,116.408456,122.761349,132.331115,135.762931,146.486768,156.269210,156.328906,157.330772,165.331981
4,Angola,Fossil CO2 Emissions,All Type,A,8.952848,8.538195,10.388669,11.371827,11.832747,10.929404,...,21.491206,24.018474,24.983535,25.221763,27.426121,29.764167,32.737569,33.548279,32.052138,30.047294


In [333]:
# Verify null values 
carbon_data_df.isnull().sum()

Location               0
Indicator              0
Type                   0
Frequency              0
1970-01-01 00:00:00    0
1971-01-01 00:00:00    0
1972-01-01 00:00:00    0
1973-01-01 00:00:00    0
1974-01-01 00:00:00    0
1975-01-01 00:00:00    0
1976-01-01 00:00:00    0
1977-01-01 00:00:00    0
1978-01-01 00:00:00    0
1979-01-01 00:00:00    0
1980-01-01 00:00:00    0
1981-01-01 00:00:00    0
1982-01-01 00:00:00    0
1983-01-01 00:00:00    0
1984-01-01 00:00:00    0
1985-01-01 00:00:00    0
1986-01-01 00:00:00    0
1987-01-01 00:00:00    0
1988-01-01 00:00:00    0
1989-01-01 00:00:00    0
1990-01-01 00:00:00    0
1991-01-01 00:00:00    0
1992-01-01 00:00:00    0
1993-01-01 00:00:00    0
1994-01-01 00:00:00    0
1995-01-01 00:00:00    0
1996-01-01 00:00:00    0
1997-01-01 00:00:00    0
1998-01-01 00:00:00    0
1999-01-01 00:00:00    0
2000-01-01 00:00:00    0
2001-01-01 00:00:00    0
2002-01-01 00:00:00    0
2003-01-01 00:00:00    0
2004-01-01 00:00:00    0
2005-01-01 00:00:00    0


In [332]:
# Check data types 
carbon_data_df.dtypes

Location                object
Indicator               object
Type                    object
Frequency               object
1970-01-01 00:00:00    float64
1971-01-01 00:00:00    float64
1972-01-01 00:00:00    float64
1973-01-01 00:00:00    float64
1974-01-01 00:00:00    float64
1975-01-01 00:00:00    float64
1976-01-01 00:00:00    float64
1977-01-01 00:00:00    float64
1978-01-01 00:00:00    float64
1979-01-01 00:00:00    float64
1980-01-01 00:00:00    float64
1981-01-01 00:00:00    float64
1982-01-01 00:00:00    float64
1983-01-01 00:00:00    float64
1984-01-01 00:00:00    float64
1985-01-01 00:00:00    float64
1986-01-01 00:00:00    float64
1987-01-01 00:00:00    float64
1988-01-01 00:00:00    float64
1989-01-01 00:00:00    float64
1990-01-01 00:00:00    float64
1991-01-01 00:00:00    float64
1992-01-01 00:00:00    float64
1993-01-01 00:00:00    float64
1994-01-01 00:00:00    float64
1995-01-01 00:00:00    float64
1996-01-01 00:00:00    float64
1997-01-01 00:00:00    float64
1998-01-

In [334]:
carbon_data_df.columns

Index(['Location', 'Indicator', 'Type', 'Frequency', '1970-01-01 00:00:00',
       '1971-01-01 00:00:00', '1972-01-01 00:00:00', '1973-01-01 00:00:00',
       '1974-01-01 00:00:00', '1975-01-01 00:00:00', '1976-01-01 00:00:00',
       '1977-01-01 00:00:00', '1978-01-01 00:00:00', '1979-01-01 00:00:00',
       '1980-01-01 00:00:00', '1981-01-01 00:00:00', '1982-01-01 00:00:00',
       '1983-01-01 00:00:00', '1984-01-01 00:00:00', '1985-01-01 00:00:00',
       '1986-01-01 00:00:00', '1987-01-01 00:00:00', '1988-01-01 00:00:00',
       '1989-01-01 00:00:00', '1990-01-01 00:00:00', '1991-01-01 00:00:00',
       '1992-01-01 00:00:00', '1993-01-01 00:00:00', '1994-01-01 00:00:00',
       '1995-01-01 00:00:00', '1996-01-01 00:00:00', '1997-01-01 00:00:00',
       '1998-01-01 00:00:00', '1999-01-01 00:00:00', '2000-01-01 00:00:00',
       '2001-01-01 00:00:00', '2002-01-01 00:00:00', '2003-01-01 00:00:00',
       '2004-01-01 00:00:00', '2005-01-01 00:00:00', '2006-01-01 00:00:00',
       '2007

In [335]:
# Convert object type to string 
# No need to convert: In pandas, strings are stored as objects
#  Pandas uses object as the datatype for strings so that you can perform size-changing 
# manipulations on the strings (e.g., concatenating them with other strings) without having 
# to recreate the entire column with a new string length.
# carbon_data_df['Location '] = carbon_data_df['Location '].astype('str')
# carbon_data_df['Indicator'] = carbon_data_df['Indicator'].astype('str')
# carbon_data_df['Type'] = carbon_data_df['Type'].astype('str')
# carbon_data_df['Frequency'] = carbon_data_df['Frequency'].astype('str')

In [336]:
# Read saved CSV 2019 data
carbon2019_df = pd.read_csv('../ETL_project/Data/CSV_data/carbon2019_csv')
carbon2019_df.head()


,Location,2019-01-01 00:00:00
0,China,9622.14
1,United States,5076.20
2,India,2525.08
3,Russia,1530.60
4,Japan,1150.06


In [337]:
# check for null values 
carbon2019_df.isnull().sum()

Location               0
2019-01-01 00:00:00    0
dtype: int64

In [338]:
# Check data types of dataframe values 
carbon2019_df.dtypes

Location                object
2019-01-01 00:00:00    float64
dtype: object

In [339]:
# Add Indicator, type, and frequency to Carbon 2019 data
carbon2019_df['Indicator']='Fossil CO2 Emissions'
carbon2019_df['Type']='All Type'
carbon2019_df['Frequency']='A'

In [340]:
carbon2019_df

,Location,2019-01-01 00:00:00,Indicator,Type,Frequency
0,China,9622.14,Fossil CO2 Emissions,All Type,A
1,United States,5076.20,Fossil CO2 Emissions,All Type,A
2,India,2525.08,Fossil CO2 Emissions,All Type,A
3,Russia,1530.60,Fossil CO2 Emissions,All Type,A
4,Japan,1150.06,Fossil CO2 Emissions,All Type,A
...,...,...,...,...,...
61,Denmark,34.08,Fossil CO2 Emissions,All Type,A
62,Slovakia,32.29,Fossil CO2 Emissions,All Type,A
63,Azerbaijan,32.14,Fossil CO2 Emissions,All Type,A
64,Trinidad and Tobago,20.96,Fossil CO2 Emissions,All Type,A


In [341]:
# Reorder Carbon 2019 dataframe
carbon2019_data_df = pd.DataFrame(carbon2019_df, columns=['Location','Indicator','Type','Frequency', '2019-01-01 00:00:00'])
carbon2019_data_df

,Location,Indicator,Type,Frequency,2019-01-01 00:00:00
0,China,Fossil CO2 Emissions,All Type,A,9622.14
1,United States,Fossil CO2 Emissions,All Type,A,5076.20
2,India,Fossil CO2 Emissions,All Type,A,2525.08
3,Russia,Fossil CO2 Emissions,All Type,A,1530.60
4,Japan,Fossil CO2 Emissions,All Type,A,1150.06
...,...,...,...,...,...
61,Denmark,Fossil CO2 Emissions,All Type,A,34.08
62,Slovakia,Fossil CO2 Emissions,All Type,A,32.29
63,Azerbaijan,Fossil CO2 Emissions,All Type,A,32.14
64,Trinidad and Tobago,Fossil CO2 Emissions,All Type,A,20.96


# Append datasets

In this section, all datasets gathered are merged into one dataframe. 
The datasets gathered from various sources do not exactly have identical list of countries. Therefore, some of the countries are not included in the final dataframe in order to avoid the existence of null values in the final clean dataset

In [302]:
# DataFrame not used: serves as a test
# # Merge population data with carbon 1970 to 2018 data for 208 countries 
# data_208countries = pd.merge(carbon_data_df, population_df, on=["Location"])
# data_208countries['Unit']="Metric Ton"
# data_208countries=data_208countries.reset_index()

# # Rearrange columns 
# data_208countries = pd.DataFrame(data_208countries, columns= ['Location','Population(2019)', 'Indicator', 'Type', 'Frequency', 'Unit', '1970-01-01 00:00:00',
#        '1971-01-01 00:00:00', '1972-01-01 00:00:00', '1973-01-01 00:00:00',
#        '1974-01-01 00:00:00', '1975-01-01 00:00:00', '1976-01-01 00:00:00',
#        '1977-01-01 00:00:00', '1978-01-01 00:00:00', '1979-01-01 00:00:00',
#        '1980-01-01 00:00:00', '1981-01-01 00:00:00', '1982-01-01 00:00:00',
#        '1983-01-01 00:00:00', '1984-01-01 00:00:00', '1985-01-01 00:00:00',
#        '1986-01-01 00:00:00', '1987-01-01 00:00:00', '1988-01-01 00:00:00',
#        '1989-01-01 00:00:00', '1990-01-01 00:00:00', '1991-01-01 00:00:00',
#        '1992-01-01 00:00:00', '1993-01-01 00:00:00', '1994-01-01 00:00:00',
#        '1995-01-01 00:00:00', '1996-01-01 00:00:00', '1997-01-01 00:00:00',
#        '1998-01-01 00:00:00', '1999-01-01 00:00:00', '2000-01-01 00:00:00',
#        '2001-01-01 00:00:00', '2002-01-01 00:00:00', '2003-01-01 00:00:00',
#        '2004-01-01 00:00:00', '2005-01-01 00:00:00', '2006-01-01 00:00:00',
#        '2007-01-01 00:00:00', '2008-01-01 00:00:00', '2009-01-01 00:00:00',
#        '2010-01-01 00:00:00', '2011-01-01 00:00:00', '2012-01-01 00:00:00',
#        '2013-01-01 00:00:00', '2014-01-01 00:00:00', '2015-01-01 00:00:00',
#        '2016-01-01 00:00:00', '2017-01-01 00:00:00', '2018-01-01 00:00:00'])


# data_208countries =data_208countries.set_index(['Location','Population(2019)','Indicator','Type', 'Frequency', 'Unit'])

# data_208countries=data_208countries.rename(columns= lambda x : str(x)[:-15])
# data_208countries=data_208countries.reset_index()
# data_208countries['Frequency']="Annually"

# data_208countries

,Location,Population(2019),Indicator,Type,Frequency,Unit,1970,1971,1972,1973,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,United States,329064.917,Fossil CO2 Emissions,All Type,Annually,Metric Ton,4688.522395,4559.408967,4807.030715,4994.593321,...,5314.842145,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845
1,Afghanistan,38041.754,Fossil CO2 Emissions,All Type,Annually,Metric Ton,1.466159,1.465293,1.510878,1.473702,...,4.248142,5.641922,6.923214,9.662172,16.311808,10.329014,9.800058,9.938866,10.187785,10.476969
2,Albania,2880.917,Fossil CO2 Emissions,All Type,Annually,Metric Ton,4.377893,4.369894,4.953242,4.540402,...,4.308789,4.592990,4.988055,4.543900,4.657987,5.064173,4.709439,4.689464,4.888502,4.802468
3,Algeria,43053.054,Fossil CO2 Emissions,All Type,Annually,Metric Ton,18.925117,22.310483,34.368764,46.469252,...,114.677988,116.408456,122.761349,132.331115,135.762931,146.486768,156.269210,156.328906,157.330772,165.331981
4,Angola,31825.295,Fossil CO2 Emissions,All Type,Annually,Metric Ton,8.952848,8.538195,10.388669,11.371827,...,21.491206,24.018474,24.983535,25.221763,27.426121,29.764167,32.737569,33.548279,32.052138,30.047294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,Vietnam,96462.106,Fossil CO2 Emissions,All Type,Annually,Metric Ton,13.407028,13.342814,14.358230,14.535535,...,138.730752,154.679966,154.562733,153.242675,159.428876,174.941642,202.770459,242.065191,242.460757,271.474096
183,Western Sahara,582.463,Fossil CO2 Emissions,All Type,Annually,Metric Ton,0.073311,0.073310,0.072442,0.074264,...,0.258750,0.265536,0.312064,0.325751,0.267194,0.242155,0.253520,0.258331,0.273070,0.283864
184,Yemen,29161.922,Fossil CO2 Emissions,All Type,Annually,Metric Ton,1.501672,1.514800,1.792100,2.209955,...,29.024687,27.175280,22.845059,21.518648,28.352236,26.911149,14.286612,11.410842,11.542397,11.767991
185,Zambia,17861.030,Fossil CO2 Emissions,All Type,Annually,Metric Ton,3.669060,3.826292,4.234740,4.539008,...,2.036439,2.239055,2.552793,3.579864,3.801032,4.275357,4.442017,4.700361,5.054812,5.166280


### Merge 2019 with 1970-2018 CO2 data

The number of countries listed in the 2019 dataset is less than the countries listed in the 1970-2019 CO2 dataset. 

In [342]:
# Set 1st carbon data (1970-2018) index 
carbon_data_df.set_index("Location", inplace=True)
carbon_data_df 

,Indicator,Type,Frequency,1970-01-01 00:00:00,1971-01-01 00:00:00,1972-01-01 00:00:00,1973-01-01 00:00:00,1974-01-01 00:00:00,1975-01-01 00:00:00,1976-01-01 00:00:00,...,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00
Location,,,,,,,,,,,,,,,,,,,,,
United States,Fossil CO2 Emissions,All Type,A,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,4907.271162,...,5314.842145,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845
Afghanistan,Fossil CO2 Emissions,All Type,A,1.466159,1.465293,1.510878,1.473702,1.895983,1.693211,1.609170,...,4.248142,5.641922,6.923214,9.662172,16.311808,10.329014,9.800058,9.938866,10.187785,10.476969
Albania,Fossil CO2 Emissions,All Type,A,4.377893,4.369894,4.953242,4.540402,4.744196,4.888888,5.256256,...,4.308789,4.592990,4.988055,4.543900,4.657987,5.064173,4.709439,4.689464,4.888502,4.802468
Algeria,Fossil CO2 Emissions,All Type,A,18.925117,22.310483,34.368764,46.469252,39.294911,35.704459,46.316389,...,114.677988,116.408456,122.761349,132.331115,135.762931,146.486768,156.269210,156.328906,157.330772,165.331981
Angola,Fossil CO2 Emissions,All Type,A,8.952848,8.538195,10.388669,11.371827,11.832747,10.929404,7.316239,...,21.491206,24.018474,24.983535,25.221763,27.426121,29.764167,32.737569,33.548279,32.052138,30.047294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,Fossil CO2 Emissions,All Type,A,13.407028,13.342814,14.358230,14.535535,9.833613,9.885531,6.047064,...,138.730752,154.679966,154.562733,153.242675,159.428876,174.941642,202.770459,242.065191,242.460757,271.474096
Western Sahara,Fossil CO2 Emissions,All Type,A,0.073311,0.073310,0.072442,0.074264,0.078865,0.082973,0.082878,...,0.258750,0.265536,0.312064,0.325751,0.267194,0.242155,0.253520,0.258331,0.273070,0.283864
Yemen,Fossil CO2 Emissions,All Type,A,1.501672,1.514800,1.792100,2.209955,2.195514,2.102490,2.562972,...,29.024687,27.175280,22.845059,21.518648,28.352236,26.911149,14.286612,11.410842,11.542397,11.767991


In [343]:
# Extract 66 countries (included in the 2019 data) from 1970-2018 data(208 countries)
reduced_countries_df = carbon_data_df.loc[['China', 'United States', 'India', 'Russia', 'Japan', 'Germany',
       'South Korea', 'Iran', 'Saudi Arabia', 'Canada', 'Indonesia', 'Mexico',
       'Brazil', 'South Africa', 'Australia', 'Turkey', 'United Kingdom',
       'Italy', 'Poland', 'France', 'Thailand', 'Spain', 'Taiwan',
       'United Arab Emirates', 'Malaysia', 'Kazakhstan', 'Singapore', 'Egypt',
       'Vietnam', 'Netherlands', 'Pakistan', 'Argentina', 'Ukraine', 'Algeria',
       'Philippines', 'Belgium', 'Venezuela', 'Qatar', 'Uzbekistan',
       'Czech Republic', 'Kuwait', 'Colombia', 'Chile', 'Bangladesh',
       'Turkmenistan', 'Greece', 'Israel', 'Romania', 'Austria', 'Belarus',
       'Portugal', 'Peru', 'Hungary', 'Finland', 'Sweden', 'Bulgaria',
       'Ireland', 'Ecuador', 'Switzerland', 'New Zealand', 'Norway', 'Denmark',
       'Slovakia', 'Azerbaijan', 'Trinidad and Tobago', 'Lithuania']]

reduced_countries_df

C:\Users\kayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  del sys.path[0]


,Indicator,Type,Frequency,1970-01-01 00:00:00,1971-01-01 00:00:00,1972-01-01 00:00:00,1973-01-01 00:00:00,1974-01-01 00:00:00,1975-01-01 00:00:00,1976-01-01 00:00:00,...,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00
Location,,,,,,,,,,,,,,,,,,,,,
China,Fossil CO2 Emissions,All Type,A,905.871699,909.657767,970.674022,1011.185712,1029.343699,1179.733887,1227.090726,...,8366.143973,9126.942721,10026.720144,10259.090913,10718.579739,10836.458234,10820.801089,10966.733668,11087.005161,11255.878289
United States,Fossil CO2 Emissions,All Type,A,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,4907.271162,...,5314.842145,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845
India,Fossil CO2 Emissions,All Type,A,232.115878,233.013554,242.324830,240.799639,258.532045,275.995901,294.960588,...,1663.191360,1750.563748,1847.213865,1985.926414,2024.083101,2218.042197,2286.820608,2371.086043,2445.882929,2621.918615
Russia,Fossil CO2 Emissions,All Type,A,1314.168301,1315.623251,1379.089133,1457.188042,1529.017739,1635.475873,1690.020667,...,1564.976584,1664.667861,1748.016660,1758.256632,1708.422047,1693.057387,1694.498309,1687.271299,1688.414681,1748.349771
Japan,Fossil CO2 Emissions,All Type,A,857.795324,855.642957,901.457823,1015.392413,1013.882856,958.157571,988.552208,...,1146.227128,1197.422042,1249.789204,1289.296188,1313.474193,1273.299828,1227.728580,1219.489173,1219.686103,1198.546373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Denmark,Fossil CO2 Emissions,All Type,A,62.443471,58.441542,60.507407,59.578422,53.202612,55.516553,61.244247,...,48.926973,49.061121,44.010921,38.829033,40.654686,36.497259,34.043938,35.471251,33.105577,33.130744
Slovakia,Fossil CO2 Emissions,All Type,A,38.452154,38.417832,39.154415,40.671493,41.491123,42.898723,43.697792,...,36.928702,39.628203,37.888613,36.398620,36.518443,34.041889,34.604879,35.181320,37.889021,38.088038
Azerbaijan,Fossil CO2 Emissions,All Type,A,35.192170,35.330777,37.655766,41.161571,44.423882,49.307598,51.828712,...,25.941210,24.745735,28.002949,30.650944,31.520291,32.976322,32.702103,33.223468,31.655183,34.704281


In [344]:
new_df = reduced_countries_df.reset_index()
new_df

,Location,Indicator,Type,Frequency,1970-01-01 00:00:00,1971-01-01 00:00:00,1972-01-01 00:00:00,1973-01-01 00:00:00,1974-01-01 00:00:00,1975-01-01 00:00:00,...,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00
0,China,Fossil CO2 Emissions,All Type,A,905.871699,909.657767,970.674022,1011.185712,1029.343699,1179.733887,...,8366.143973,9126.942721,10026.720144,10259.090913,10718.579739,10836.458234,10820.801089,10966.733668,11087.005161,11255.878289
1,United States,Fossil CO2 Emissions,All Type,A,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,...,5314.842145,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845
2,India,Fossil CO2 Emissions,All Type,A,232.115878,233.013554,242.324830,240.799639,258.532045,275.995901,...,1663.191360,1750.563748,1847.213865,1985.926414,2024.083101,2218.042197,2286.820608,2371.086043,2445.882929,2621.918615
3,Russia,Fossil CO2 Emissions,All Type,A,1314.168301,1315.623251,1379.089133,1457.188042,1529.017739,1635.475873,...,1564.976584,1664.667861,1748.016660,1758.256632,1708.422047,1693.057387,1694.498309,1687.271299,1688.414681,1748.349771
4,Japan,Fossil CO2 Emissions,All Type,A,857.795324,855.642957,901.457823,1015.392413,1013.882856,958.157571,...,1146.227128,1197.422042,1249.789204,1289.296188,1313.474193,1273.299828,1227.728580,1219.489173,1219.686103,1198.546373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Denmark,Fossil CO2 Emissions,All Type,A,62.443471,58.441542,60.507407,59.578422,53.202612,55.516553,...,48.926973,49.061121,44.010921,38.829033,40.654686,36.497259,34.043938,35.471251,33.105577,33.130744
62,Slovakia,Fossil CO2 Emissions,All Type,A,38.452154,38.417832,39.154415,40.671493,41.491123,42.898723,...,36.928702,39.628203,37.888613,36.398620,36.518443,34.041889,34.604879,35.181320,37.889021,38.088038
63,Azerbaijan,Fossil CO2 Emissions,All Type,A,35.192170,35.330777,37.655766,41.161571,44.423882,49.307598,...,25.941210,24.745735,28.002949,30.650944,31.520291,32.976322,32.702103,33.223468,31.655183,34.704281
64,Trinidad and Tobago,Fossil CO2 Emissions,All Type,A,6.346294,6.311688,6.418081,6.682062,6.505548,5.897179,...,34.494760,38.716016,38.696049,33.558319,35.531630,37.051541,36.356837,35.944218,35.976914,35.950703


In [345]:
# Append/Merge Dataframe
#combined_co2_df = pd.merge(new_df, carbon2019_data_df, on=["Location", "Indicator", "Type", "Frequency"])
combined_co2_df = pd.merge(new_df, carbon2019_data_df, on=["Location"])

# Delete unwanted columns
del combined_co2_df['Frequency_y']
del combined_co2_df['Type_y']
del combined_co2_df['Indicator_y']

# Rename columns
combined_co2 = combined_co2_df.rename(columns={'Indicator_x': "Indicator", 'Type_x': 'Type', 'Frequency_x':'Frequency'})

# Change Frequency to Annually (Remove Abbreviation to make it clearer)
combined_co2['Frequency']='Annually'

combined_co2

,Location,Indicator,Type,Frequency,1970-01-01 00:00:00,1971-01-01 00:00:00,1972-01-01 00:00:00,1973-01-01 00:00:00,1974-01-01 00:00:00,1975-01-01 00:00:00,...,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00,2019-01-01 00:00:00
0,China,Fossil CO2 Emissions,All Type,Annually,905.871699,909.657767,970.674022,1011.185712,1029.343699,1179.733887,...,9126.942721,10026.720144,10259.090913,10718.579739,10836.458234,10820.801089,10966.733668,11087.005161,11255.878289,9622.14
1,United States,Fossil CO2 Emissions,All Type,Annually,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,...,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845,5076.20
2,India,Fossil CO2 Emissions,All Type,Annually,232.115878,233.013554,242.324830,240.799639,258.532045,275.995901,...,1750.563748,1847.213865,1985.926414,2024.083101,2218.042197,2286.820608,2371.086043,2445.882929,2621.918615,2525.08
3,Russia,Fossil CO2 Emissions,All Type,Annually,1314.168301,1315.623251,1379.089133,1457.188042,1529.017739,1635.475873,...,1664.667861,1748.016660,1758.256632,1708.422047,1693.057387,1694.498309,1687.271299,1688.414681,1748.349771,1530.60
4,Japan,Fossil CO2 Emissions,All Type,Annually,857.795324,855.642957,901.457823,1015.392413,1013.882856,958.157571,...,1197.422042,1249.789204,1289.296188,1313.474193,1273.299828,1227.728580,1219.489173,1219.686103,1198.546373,1150.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Denmark,Fossil CO2 Emissions,All Type,Annually,62.443471,58.441542,60.507407,59.578422,53.202612,55.516553,...,49.061121,44.010921,38.829033,40.654686,36.497259,34.043938,35.471251,33.105577,33.130744,34.08
62,Slovakia,Fossil CO2 Emissions,All Type,Annually,38.452154,38.417832,39.154415,40.671493,41.491123,42.898723,...,39.628203,37.888613,36.398620,36.518443,34.041889,34.604879,35.181320,37.889021,38.088038,32.29
63,Azerbaijan,Fossil CO2 Emissions,All Type,Annually,35.192170,35.330777,37.655766,41.161571,44.423882,49.307598,...,24.745735,28.002949,30.650944,31.520291,32.976322,32.702103,33.223468,31.655183,34.704281,32.14
64,Trinidad and Tobago,Fossil CO2 Emissions,All Type,Annually,6.346294,6.311688,6.418081,6.682062,6.505548,5.897179,...,38.716016,38.696049,33.558319,35.531630,37.051541,36.356837,35.944218,35.976914,35.950703,20.96


In [347]:
# Reset index in order to rename date columns to year 
combined_co2_df =combined_co2.set_index(['Location','Indicator','Type', 'Frequency'])
combined_co2_df

,,,,1970-01-01 00:00:00,1971-01-01 00:00:00,1972-01-01 00:00:00,1973-01-01 00:00:00,1974-01-01 00:00:00,1975-01-01 00:00:00,1976-01-01 00:00:00,1977-01-01 00:00:00,1978-01-01 00:00:00,1979-01-01 00:00:00,...,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00,2019-01-01 00:00:00
Location,Indicator,Type,Frequency,,,,,,,,,,,,,,,,,,,,,
China,Fossil CO2 Emissions,All Type,Annually,905.871699,909.657767,970.674022,1011.185712,1029.343699,1179.733887,1227.090726,1379.490057,1555.775630,1593.621377,...,9126.942721,10026.720144,10259.090913,10718.579739,10836.458234,10820.801089,10966.733668,11087.005161,11255.878289,9622.14
United States,Fossil CO2 Emissions,All Type,Annually,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,4907.271162,5071.365566,5066.244675,5105.068431,...,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845,5076.20
India,Fossil CO2 Emissions,All Type,Annually,232.115878,233.013554,242.324830,240.799639,258.532045,275.995901,294.960588,299.905747,267.423683,287.477109,...,1750.563748,1847.213865,1985.926414,2024.083101,2218.042197,2286.820608,2371.086043,2445.882929,2621.918615,2525.08
Russia,Fossil CO2 Emissions,All Type,Annually,1314.168301,1315.623251,1379.089133,1457.188042,1529.017739,1635.475873,1690.020667,1727.377704,1833.384816,1855.228230,...,1664.667861,1748.016660,1758.256632,1708.422047,1693.057387,1694.498309,1687.271299,1688.414681,1748.349771,1530.60
Japan,Fossil CO2 Emissions,All Type,Annually,857.795324,855.642957,901.457823,1015.392413,1013.882856,958.157571,988.552208,1013.479852,1013.490477,1039.200847,...,1197.422042,1249.789204,1289.296188,1313.474193,1273.299828,1227.728580,1219.489173,1219.686103,1198.546373,1150.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Denmark,Fossil CO2 Emissions,All Type,Annually,62.443471,58.441542,60.507407,59.578422,53.202612,55.516553,61.244247,62.973987,62.934926,66.374163,...,49.061121,44.010921,38.829033,40.654686,36.497259,34.043938,35.471251,33.105577,33.130744,34.08
Slovakia,Fossil CO2 Emissions,All Type,Annually,38.452154,38.417832,39.154415,40.671493,41.491123,42.898723,43.697792,45.409827,57.381835,58.407022,...,39.628203,37.888613,36.398620,36.518443,34.041889,34.604879,35.181320,37.889021,38.088038,32.29
Azerbaijan,Fossil CO2 Emissions,All Type,Annually,35.192170,35.330777,37.655766,41.161571,44.423882,49.307598,51.828712,54.478499,57.262258,57.226205,...,24.745735,28.002949,30.650944,31.520291,32.976322,32.702103,33.223468,31.655183,34.704281,32.14


In [348]:
# Rename date columns: only show year 
combined_co2_df=combined_co2_df.rename(columns= lambda x : str(x)[:-15])
combined_co2_df

,,,,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,Indicator,Type,Frequency,,,,,,,,,,,,,,,,,,,,,
China,Fossil CO2 Emissions,All Type,Annually,905.871699,909.657767,970.674022,1011.185712,1029.343699,1179.733887,1227.090726,1379.490057,1555.775630,1593.621377,...,9126.942721,10026.720144,10259.090913,10718.579739,10836.458234,10820.801089,10966.733668,11087.005161,11255.878289,9622.14
United States,Fossil CO2 Emissions,All Type,Annually,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,4907.271162,5071.365566,5066.244675,5105.068431,...,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845,5076.20
India,Fossil CO2 Emissions,All Type,Annually,232.115878,233.013554,242.324830,240.799639,258.532045,275.995901,294.960588,299.905747,267.423683,287.477109,...,1750.563748,1847.213865,1985.926414,2024.083101,2218.042197,2286.820608,2371.086043,2445.882929,2621.918615,2525.08
Russia,Fossil CO2 Emissions,All Type,Annually,1314.168301,1315.623251,1379.089133,1457.188042,1529.017739,1635.475873,1690.020667,1727.377704,1833.384816,1855.228230,...,1664.667861,1748.016660,1758.256632,1708.422047,1693.057387,1694.498309,1687.271299,1688.414681,1748.349771,1530.60
Japan,Fossil CO2 Emissions,All Type,Annually,857.795324,855.642957,901.457823,1015.392413,1013.882856,958.157571,988.552208,1013.479852,1013.490477,1039.200847,...,1197.422042,1249.789204,1289.296188,1313.474193,1273.299828,1227.728580,1219.489173,1219.686103,1198.546373,1150.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Denmark,Fossil CO2 Emissions,All Type,Annually,62.443471,58.441542,60.507407,59.578422,53.202612,55.516553,61.244247,62.973987,62.934926,66.374163,...,49.061121,44.010921,38.829033,40.654686,36.497259,34.043938,35.471251,33.105577,33.130744,34.08
Slovakia,Fossil CO2 Emissions,All Type,Annually,38.452154,38.417832,39.154415,40.671493,41.491123,42.898723,43.697792,45.409827,57.381835,58.407022,...,39.628203,37.888613,36.398620,36.518443,34.041889,34.604879,35.181320,37.889021,38.088038,32.29
Azerbaijan,Fossil CO2 Emissions,All Type,Annually,35.192170,35.330777,37.655766,41.161571,44.423882,49.307598,51.828712,54.478499,57.262258,57.226205,...,24.745735,28.002949,30.650944,31.520291,32.976322,32.702103,33.223468,31.655183,34.704281,32.14


In [349]:
# Reset index back after renaming 
combined_co2_df=combined_co2_df.reset_index()
combined_co2_df

,Location,Indicator,Type,Frequency,1970,1971,1972,1973,1974,1975,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,China,Fossil CO2 Emissions,All Type,Annually,905.871699,909.657767,970.674022,1011.185712,1029.343699,1179.733887,...,9126.942721,10026.720144,10259.090913,10718.579739,10836.458234,10820.801089,10966.733668,11087.005161,11255.878289,9622.14
1,United States,Fossil CO2 Emissions,All Type,Annually,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,4612.546577,...,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845,5076.20
2,India,Fossil CO2 Emissions,All Type,Annually,232.115878,233.013554,242.324830,240.799639,258.532045,275.995901,...,1750.563748,1847.213865,1985.926414,2024.083101,2218.042197,2286.820608,2371.086043,2445.882929,2621.918615,2525.08
3,Russia,Fossil CO2 Emissions,All Type,Annually,1314.168301,1315.623251,1379.089133,1457.188042,1529.017739,1635.475873,...,1664.667861,1748.016660,1758.256632,1708.422047,1693.057387,1694.498309,1687.271299,1688.414681,1748.349771,1530.60
4,Japan,Fossil CO2 Emissions,All Type,Annually,857.795324,855.642957,901.457823,1015.392413,1013.882856,958.157571,...,1197.422042,1249.789204,1289.296188,1313.474193,1273.299828,1227.728580,1219.489173,1219.686103,1198.546373,1150.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Denmark,Fossil CO2 Emissions,All Type,Annually,62.443471,58.441542,60.507407,59.578422,53.202612,55.516553,...,49.061121,44.010921,38.829033,40.654686,36.497259,34.043938,35.471251,33.105577,33.130744,34.08
62,Slovakia,Fossil CO2 Emissions,All Type,Annually,38.452154,38.417832,39.154415,40.671493,41.491123,42.898723,...,39.628203,37.888613,36.398620,36.518443,34.041889,34.604879,35.181320,37.889021,38.088038,32.29
63,Azerbaijan,Fossil CO2 Emissions,All Type,Annually,35.192170,35.330777,37.655766,41.161571,44.423882,49.307598,...,24.745735,28.002949,30.650944,31.520291,32.976322,32.702103,33.223468,31.655183,34.704281,32.14
64,Trinidad and Tobago,Fossil CO2 Emissions,All Type,Annually,6.346294,6.311688,6.418081,6.682062,6.505548,5.897179,...,38.716016,38.696049,33.558319,35.531630,37.051541,36.356837,35.944218,35.976914,35.950703,20.96


In [350]:
# Add population data to the final combined df 
# Extract 66 countries in 1970-2018 data(208 countries)
population_df= population_df.set_index('Location')

reduced_population_df = population_df.loc[['China', 'United States', 'India', 'Russia', 'Japan', 'Germany',
       'South Korea', 'Iran', 'Saudi Arabia', 'Canada', 'Indonesia', 'Mexico',
       'Brazil', 'South Africa', 'Australia', 'Turkey', 'United Kingdom',
       'Italy', 'Poland', 'France', 'Thailand', 'Spain', 'Taiwan',
       'United Arab Emirates', 'Malaysia', 'Kazakhstan', 'Singapore', 'Egypt',
       'Vietnam', 'Netherlands', 'Pakistan', 'Argentina', 'Ukraine', 'Algeria',
       'Philippines', 'Belgium', 'Venezuela', 'Qatar', 'Uzbekistan',
       'Czech Republic', 'Kuwait', 'Colombia', 'Chile', 'Bangladesh',
       'Turkmenistan', 'Greece', 'Israel', 'Romania', 'Austria', 'Belarus',
       'Portugal', 'Peru', 'Hungary', 'Finland', 'Sweden', 'Bulgaria',
       'Ireland', 'Ecuador', 'Switzerland', 'New Zealand', 'Norway', 'Denmark',
       'Slovakia', 'Azerbaijan', 'Trinidad and Tobago', 'Lithuania']]

reduced_population_df


,Population(2019)
Location,
China,1433783.686
United States,329064.917
India,1366417.754
Russia,145872.256
Japan,126860.301
...,...
Denmark,5771.876
Slovakia,5457.013
Azerbaijan,10047.718


In [351]:
# Merge population to co2 df 
final_combined_df = pd.merge(reduced_population_df,combined_co2_df, on=["Location"])

# Rename column 
final_combined_df=final_combined_df.rename(columns={"Location":"Country"})

In [352]:
# Display final dataframe
final_combined_df

,Country,Population(2019),Indicator,Type,Frequency,1970,1971,1972,1973,1974,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,China,1433783.686,Fossil CO2 Emissions,All Type,Annually,905.871699,909.657767,970.674022,1011.185712,1029.343699,...,9126.942721,10026.720144,10259.090913,10718.579739,10836.458234,10820.801089,10966.733668,11087.005161,11255.878289,9622.14
1,United States,329064.917,Fossil CO2 Emissions,All Type,Annually,4688.522395,4559.408967,4807.030715,4994.593321,4827.419694,...,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845,5076.20
2,India,1366417.754,Fossil CO2 Emissions,All Type,Annually,232.115878,233.013554,242.324830,240.799639,258.532045,...,1750.563748,1847.213865,1985.926414,2024.083101,2218.042197,2286.820608,2371.086043,2445.882929,2621.918615,2525.08
3,Russia,145872.256,Fossil CO2 Emissions,All Type,Annually,1314.168301,1315.623251,1379.089133,1457.188042,1529.017739,...,1664.667861,1748.016660,1758.256632,1708.422047,1693.057387,1694.498309,1687.271299,1688.414681,1748.349771,1530.60
4,Japan,126860.301,Fossil CO2 Emissions,All Type,Annually,857.795324,855.642957,901.457823,1015.392413,1013.882856,...,1197.422042,1249.789204,1289.296188,1313.474193,1273.299828,1227.728580,1219.489173,1219.686103,1198.546373,1150.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Denmark,5771.876,Fossil CO2 Emissions,All Type,Annually,62.443471,58.441542,60.507407,59.578422,53.202612,...,49.061121,44.010921,38.829033,40.654686,36.497259,34.043938,35.471251,33.105577,33.130744,34.08
62,Slovakia,5457.013,Fossil CO2 Emissions,All Type,Annually,38.452154,38.417832,39.154415,40.671493,41.491123,...,39.628203,37.888613,36.398620,36.518443,34.041889,34.604879,35.181320,37.889021,38.088038,32.29
63,Azerbaijan,10047.718,Fossil CO2 Emissions,All Type,Annually,35.192170,35.330777,37.655766,41.161571,44.423882,...,24.745735,28.002949,30.650944,31.520291,32.976322,32.702103,33.223468,31.655183,34.704281,32.14
64,Trinidad and Tobago,1394.973,Fossil CO2 Emissions,All Type,Annually,6.346294,6.311688,6.418081,6.682062,6.505548,...,38.716016,38.696049,33.558319,35.531630,37.051541,36.356837,35.944218,35.976914,35.950703,20.96


In [353]:
# Insert unit column 
final_combined_df['Unit'] = "Metric ton"


In [354]:
# Rearrange columns 
# print(final_combined_df.columns)
final_combined_df=pd.DataFrame(final_combined_df, columns=['Country', 'Population(2019)', 'Indicator', 'Type', 'Frequency', 'Unit',
                                                       '1970','1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
                                                       '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
                                                       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
                                                       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
                                                       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
                                                       '2016', '2017', '2018', '2019'])
final_combined_df

,Country,Population(2019),Indicator,Type,Frequency,Unit,1970,1971,1972,1973,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,China,1433783.686,Fossil CO2 Emissions,All Type,Annually,Metric ton,905.871699,909.657767,970.674022,1011.185712,...,9126.942721,10026.720144,10259.090913,10718.579739,10836.458234,10820.801089,10966.733668,11087.005161,11255.878289,9622.14
1,United States,329064.917,Fossil CO2 Emissions,All Type,Annually,Metric ton,4688.522395,4559.408967,4807.030715,4994.593321,...,5555.444090,5425.111739,5244.133269,5316.463259,5389.262507,5225.394249,5161.072868,5128.440907,5275.477845,5076.20
2,India,1366417.754,Fossil CO2 Emissions,All Type,Annually,Metric ton,232.115878,233.013554,242.324830,240.799639,...,1750.563748,1847.213865,1985.926414,2024.083101,2218.042197,2286.820608,2371.086043,2445.882929,2621.918615,2525.08
3,Russia,145872.256,Fossil CO2 Emissions,All Type,Annually,Metric ton,1314.168301,1315.623251,1379.089133,1457.188042,...,1664.667861,1748.016660,1758.256632,1708.422047,1693.057387,1694.498309,1687.271299,1688.414681,1748.349771,1530.60
4,Japan,126860.301,Fossil CO2 Emissions,All Type,Annually,Metric ton,857.795324,855.642957,901.457823,1015.392413,...,1197.422042,1249.789204,1289.296188,1313.474193,1273.299828,1227.728580,1219.489173,1219.686103,1198.546373,1150.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Denmark,5771.876,Fossil CO2 Emissions,All Type,Annually,Metric ton,62.443471,58.441542,60.507407,59.578422,...,49.061121,44.010921,38.829033,40.654686,36.497259,34.043938,35.471251,33.105577,33.130744,34.08
62,Slovakia,5457.013,Fossil CO2 Emissions,All Type,Annually,Metric ton,38.452154,38.417832,39.154415,40.671493,...,39.628203,37.888613,36.398620,36.518443,34.041889,34.604879,35.181320,37.889021,38.088038,32.29
63,Azerbaijan,10047.718,Fossil CO2 Emissions,All Type,Annually,Metric ton,35.192170,35.330777,37.655766,41.161571,...,24.745735,28.002949,30.650944,31.520291,32.976322,32.702103,33.223468,31.655183,34.704281,32.14
64,Trinidad and Tobago,1394.973,Fossil CO2 Emissions,All Type,Annually,Metric ton,6.346294,6.311688,6.418081,6.682062,...,38.716016,38.696049,33.558319,35.531630,37.051541,36.356837,35.944218,35.976914,35.950703,20.96


In [355]:
# Save as CSV 
final_combined_df.to_csv('../ETL_project/Data/CSV_data/final_co2_data(66_countries)')

# Save data to MongoDB

### Dataframes to save: 
* 1970-2019(66 countries: Location ordered by 2019 emission ranking)
* 1970-2018(208 countries: Location aphabetically ordered)

In [356]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [288]:
# Define database and collection
db = client.globalco2emission_dt
collection = db.CO2_66countries_1970to2019

In [357]:
# Insert final dataframe into MongoDB 
# final_combined_df.reset_index(inplace=True)
data_dictt = final_combined_df.to_dict("records")
collection.insert_many(data_dictt)


In [362]:
# Insert CO2 emission data for 208 countries (1970-2018 excluding year 2019)
carbon_data_df=carbon_data_df.reset_index()
collection = db.CO2_208countries
# final_combined_df.reset_index(inplace=True)
data_dict = carbon_data_df.to_dict("records")
collection.insert_many(data_dict)